In [9]:
import pandas as pd
import spacy
from NLP_workbench import text_preprocessing as tpp

In [10]:
data = pd.read_csv('../data/train/train.csv')
del data['id']
data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [11]:
preproc = tpp.TextPreprocessing()

In [12]:
text = data['comment_text'][0]
print(preproc.preprocess_text(text, stemming=False ,check_valid_word=False))

['explanation', 'edits', 'made', 'username', 'hardcore', 'metallica', 'fan', 'reverted', 'werent', 'vandalism', 'closure', 'gas', 'voted', 'new', 'york', 'doll', 'fac', 'please', 'dont', 'remove', 'template', 'talk', 'page', 'since', 'im', 'retired']


In [13]:
%timeit preproc.preprocess_text(data['comment_text'][0], check_valid_word=False)

261 µs ± 9.34 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
#data['comment_text'] = data['comment_text'].apply(lambda x: preproc.preprocess_text(x, check_valid_word=False))

#data

In [15]:
#data = pd.read_csv("../data/train/processed_train.csv")
#del data['Unnamed: 0']

In [16]:
def fun(x):
    if x.sum() != 0:
        return 1
    return 0

rows = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

data['hate'] = data[rows].apply(lambda x: fun(x), axis=1)

for i in rows:
    del data[i]

In [17]:
1 - data['hate'].sum()/len(data)

0.8983211235124177

In [18]:
data

,comment_text,hate
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [19]:
#data.to_csv('../data/train/preprocessed_train_clean.csv', index=False)

In [20]:
#data = pd.read_csv('../data/train/preprocessed_train_clean.csv')
#data['comment_text'].apply(eval)

In [21]:
from transformers import BertModel, BertTokenizer
from tqdm.auto import tqdm

model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [22]:
%timeit data['comment_text'][:1].apply(lambda x: tokenizer.encode_plus(x, add_special_tokens=True, max_length=512, truncation=True, return_tensors='pt'))

1.35 ms ± 178 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
data['comment_text'] = data['comment_text'].apply(lambda x: tokenizer.encode_plus(x, add_special_tokens=True, max_length=512, truncation=True, return_tensors='pt'))

In [24]:
data.to_pickle('../data/train/BERT_tokenized_data.pkl')

In [25]:
tokenized = pd.read_pickle('../data/train/BERT_tokenized_data.pkl')

In [ ]:
import torch
# Extrahieren der 'input_ids' und 'attention_mask' aus dem tokenisierten Output
input_ids = torch.cat([item['input_ids'] for item in data], dim=0)
attention_mask = torch.cat([item['attention_mask'] for item in data], dim=0)

In [4]:
# Initialisieren des BERT-Modells
model = BertModel.from_pretrained('bert-base-uncased')

# Führen Sie die Token durch das BERT-Modell und speichern Sie die letzten versteckten Zustände
last_hidden_states_batches = []

# Verwenden Sie tqdm, um eine Fortschrittsanzeige für die Verarbeitung der Batches anzuzeigen
for i in tqdm(range(0, input_ids.size(0), batch_size), desc='Processing'):
    batch_input_ids = input_ids[i:i+batch_size]
    batch_attention_mask = attention_mask[i:i+batch_size]
    
    with torch.no_grad():
        batch_outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
    
    batch_last_hidden_states = batch_outputs.last_hidden_state
    last_hidden_states_batches.append(batch_last_hidden_states)

# Kombinieren der Ergebnisse aus allen Batches in einen einzigen Tensor
last_hidden_states = torch.cat(last_hidden_states_batches, dim=0)

NameError: name 'input_ids' is not defined